In [1]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

from bs4 import BeautifulSoup

In [2]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


In [4]:
companies_path = 'CSVs/clutch_companies.csv'

In [5]:
# Clutch IT Firms URL

clutch_url = 'https://clutch.co/it-services/outsourcing' 

In [ ]:
#HTML Div class references

firms_number_html='tabs-info'
firms_list_html='company_logotype'
firm_name_html='header-company--title'
firm_score_html='ratingValue'
firm_location_html='location-name'
firm_clients_html='field-name-clients'
firm_client_name='field c'

In [8]:
def get_total_pages_at(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    firms_tab_text = soup.find("div",attrs={"class" : "tabs-info"}, recursive=True).text
    firms_number = firms_tab_text.replace(',', '').replace('Firms', '').strip()
    return math.ceil(int(firms_number) / 19)

In [9]:
def find_url_references_at(browser):
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    url_list = soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True)
    return [url.find('a')['href'] for url in url_list]

In [12]:
def get_companies_csv():
    try: return pd.read_csv(companies_path)
    except: return pd.DataFrame()

In [13]:
def collect_data_from(ref_list):
    df = get_companies_csv()
    for ref in ref_list:
        print('Collecting data from:', ref)
        data = colect_infos_at(ref)
        df = df.append(data, ignore_index=True)
    return df

In [10]:
def colect_infos_at(reference):
    browser.get(f"https://clutch.co{reference}")
    time.sleep(2)
    
    soup = BeautifulSoup(browser.page_source, features='html.parser')

    name = soup.find("h1", attrs={"class" : "header-company--title"}).text.replace('\n', '').strip()
    score = soup.find("span", attrs={"itemprop" : "ratingValue"}).text.replace('\n', '').strip()
    location = soup.find("span", attrs={"class" : "location-name"}).text.replace('\n', '').strip()
    
    clients = ''
    try:
        clients = soup.find("div", attrs={"class" : "field-name-clients"})
        clients =  ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
    except:
        try:
            clients = soup.find("div", attrs={"class" : "field c"})
            clients = ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
        except:
            print(f"CLIENTES NÃO ENCONTRADOS EM {name}")
            
    return {'name': name,
            'clutch_score': score,
            'clutch_location': location,
            'clutch_clients': clients}

In [14]:
def collect_companies():
    
    print('Accessing: ', clutch_url)
    browser.get(clutch_url)
    time.sleep(2)
    
    pages = 5 #get_total_pages_at(browser)
    print('Get: ', str(pages), 'pages')
    
    for i in range(1,pages):
        
        print('Accessing page: ', i)
        browser.get(f"https://clutch.co/it-services/outsourcing?page={i}")
        time.sleep(1)
        
        reference_list = find_url_references_at(browser)
        print('Find ', len(reference_list), 'references')
        
        clutch_df = collect_data_from(reference_list)
        clutch_df.to_csv(companies_path, index=False)
    
collect_companies()

get_companies_csv()

Accessing:  https://clutch.co/it-services/outsourcing
Get:  5 pages
Accessing page:  1
Find  20 references
CLIENTES NÃO ENCONTRADOS EM delaPlex
CLIENTES NÃO ENCONTRADOS EM Avenga
CLIENTES NÃO ENCONTRADOS EM TechMagic
CLIENTES NÃO ENCONTRADOS EM Skelia
Accessing page:  2
Find  20 references
CLIENTES NÃO ENCONTRADOS EM Alpacked
Accessing page:  3
Find  20 references
Accessing page:  4
Find  20 references
CLIENTES NÃO ENCONTRADOS EM CHI Software
CLIENTES NÃO ENCONTRADOS EM Oneture Technologies
CLIENTES NÃO ENCONTRADOS EM Evojam


,clutch_clients,clutch_location,clutch_score,name
0,NaN,"Lake Oswego, OR",4.9,"Unosquare, LLC"
1,Vivino - #1 wine app/platform in the world (Da...,"Amsterdam, Netherlands",4.8,Symphony Solutions
2,NaN,"Lake Oswego, OR",4.9,"Unosquare, LLC"
3,Vivino - #1 wine app/platform in the world (Da...,"Amsterdam, Netherlands",4.8,Symphony Solutions
4,NaN,"Lake Oswego, OR",4.9,"Unosquare, LLC"
...,...,...,...,...
91,"PwC, Beckman Coulter, Grundfos, AIDS Healthca...","Richardson, TX",4.7,Impiger Technologies
92,NaN,"Warsaw, Poland",4.9,Evojam
93,300+ midsize to large corporations including:...,"Krakow, Poland",5.0,AMC TECH
94,"GetCompliant, Homer, Rule, FundedByMe, Trailh...","Kharkov, Ukraine",5.0,Vilmate LLC


In [27]:
df_clean = get_companies_csv().drop_duplicates(subset=['name'])
df_clean = df_clean.sort_values(by=['clutch_score'], ascending=False)
df_clean

,clutch_clients,clutch_location,clutch_score,name
66,Berezha Security has built a solid client base...,"Kyiv, Ukraine",5.0,Berezha Security Group
22,"Porsche AG, Volkswagen AG, Allstate Insurance,...","Cracow, Poland",5.0,Grape Up
41,"Coca-Cola, ebay, Vodafone, JLL, SKF, Brightco...","Sunnyvale, CA",5.0,Algoworks
40,NaN,"Kyiv, Ukraine",5.0,Alpacked
44,"Redbull, Hilton, Sony Music, Boy Scouts of Am...","Orlando, FL",5.0,Simform
...,...,...,...,...
91,"PwC, Beckman Coulter, Grundfos, AIDS Healthca...","Richardson, TX",4.7,Impiger Technologies
61,Over the past 18 years we collaborated with F...,"Kyiv, Ukraine",4.7,Sigma Software
83,"Orange, ING Bank, Carrefour, Actility, Braebu...","Bucharest, Romania",4.7,Tremend Software Consulting
74,"LifeTime Fitness, Daikin, WEX Health, MTS Sys...","Minneapolis, MN",4.6,Coherent Solutions


In [28]:
browser.get("https://clutch.co/profile/pattern-match")
soup = BeautifulSoup(browser.page_source, features='html.parser')


main_div = soup.find("div", attrs={"class" : "module-list"}) 
main_div.find_all("span")

valor = ''
preco_hora = ''
funcionarios = ''

main_div = [i.text for i in main_div.find_all("span")]
main_div
valor, preco_hora, funcionarios = [main_div[i] for i in (0, 1, 2)]

print(f"valor {valor}")
print(f"preco_hora {preco_hora}")
print(f"funcionarios {funcionarios}")

valor $10,000+
preco_hora $50 - $99 / hr
funcionarios 10 - 49
